### Required Libraries

In [1]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from operator import itemgetter
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders.pdf import PyMuPDFLoader

### Defining the models

In [ ]:
Model = "llama3"; 
Embedding_Model = "mxbai-embed-large"
model = Ollama(model=Model)

### Loading data and splitting into chunks

In [ ]:
loader = DirectoryLoader(path="testDataMD",glob="*.txt")
documents = loader.load()

text_splitter = CharacterTextSplitter(separator="##", chunk_size=1000, chunk_overlap=0)
docs2 = text_splitter.split_documents(documents)

### Creating the vectorstore

In [ ]:
vectors = FAISS.from_documents(docs2, OllamaEmbeddings(model = Embedding_Model))
retriever2 = vectors.as_retriever()

### Defining the RAG Model

In [7]:
# A function to combine retrieved documents into a string 

def combineDocs(docs):
    context = "\n\n".join(f'Document [ Metadata : \n{doc.metadata} \nPage Content : \n{doc.page_content} ]' for doc in docs)
    return context

#### Method 1

In [5]:
def create_model(vectorDB):
    
    # This is where the conversation will be stored
    memory = ConversationBufferMemory()

    # Creating a conversation model for ConvChain

    def rag_retrieval(question, memory):

        # Retrieve docs from vector DB
        docs = vectorDB.similarity_search(question, k = 4)
        context = combineDocs(docs)

        conversation = memory.load_memory_variables({}).get('history', '')

        # Add context to the prompt
        prompt = f'Answer the question based on the context below and the previous conversations. If you cannot\nanswer the question, reply "Oof that\'s a tough one, i don\'t really know this"\n\nPrevious Conversation : \n{conversation}\n\nContext : \n{context}\n\nQuestion from user: {question}'

        response = model.invoke(prompt)

        memory.add_user_message(question)
        memory.add_bot_message(response)

        return response
    
    chain = ConversationChain(memory = memory, conversation_model = rag_retrieval)

    return chain

#### Method 2

In [ ]:
chatHistory = "\n\n"

template = """
You are acting as a chat-bot
Answer the user's question based on the context (provided by user as Documents) below and the previous conversations between you (AI) and the user (User). 
Utilise your understanding to see whether context is to be used or the previous conversation is to be used or both together to best answer the question
If you can't answer the question, reply "Oof that's a tough one, i don't really know this"

Conversation till now : {conversation}

Context : {context}

Question : {question}

"""

prompt = PromptTemplate.from_template(template)

def chat(question): 
    
    # Retrieve docs from vector DB
    docs = vectors.similarity_search(question, k = 4)
    contextString = combineDocs(docs)
    
    query = prompt.format(conversation = chatHistory, context = contextString, question = question)

    response = model.invoke(query)
    
    chatHistory = chatHistory + "\nUser : " + question + "\nAI : " + response

    return response
